<a href="https://colab.research.google.com/github/Debayan2004/BR-Tumor-Segmentation/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import Add, Multiply

def residual_block(x, filters, kernel_size=(3, 3, 3), l2_reg=1e-4):
    """
    A residual block with two convolutional layers and a skip connection.
    """
    shortcut = x
    x = layers.Conv3D(filters, kernel_size, activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv3D(filters, kernel_size, activation=None, padding='same',
                      kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = Add()([x, shortcut])  # Skip connection
    x = layers.Activation('relu')(x)
    return x

def attention_gate(x, g, inter_channel):
    """
    An attention gate for the skip connection.
    x: Input feature map from the encoder
    g: Gating signal from the decoder
    inter_channel: Number of intermediate channels
    """
    theta_x = layers.Conv3D(inter_channel, kernel_size=1, strides=1, padding='same')(x)
    phi_g = layers.Conv3D(inter_channel, kernel_size=1, strides=1, padding='same')(g)
    attention = layers.Add()([theta_x, phi_g])
    attention = layers.Activation('relu')(attention)
    attention = layers.Conv3D(1, kernel_size=1, strides=1, padding='same')(attention)
    attention = layers.Activation('sigmoid')(attention)
    x = Multiply()([x, attention])
    return x

def build_3d_unet_with_residual_and_attention(input_shape=(8, 16, 16, 3), num_classes=32, dropout_rate=0.3, l2_reg=1e-4):
    """
    Build a 3D U-Net with residual blocks and attention mechanisms.
    """
    inputs = layers.Input(shape=input_shape)

    # Encoder Level 1: 64 filters
    x1 = residual_block(inputs, 64, l2_reg=l2_reg)
    p1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(x1)

    # Encoder Level 2: 128 filters
    x2 = residual_block(p1, 128, l2_reg=l2_reg)
    p2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(x2)

    # Encoder Level 3: 256 filters
    x3 = residual_block(p2, 256, l2_reg=l2_reg)
    p3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(x3)

    # Encoder Level 4: 512 filters
    x4 = residual_block(p3, 512, l2_reg=l2_reg)
    p4 = layers.MaxPooling3D(pool_size=(2, 2, 2))(x4)

    # Bottleneck
    x = residual_block(p4, 1024, l2_reg=l2_reg)

    # Decoder Level 4 (up to Level 3)
    x = layers.Conv3DTranspose(512, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    x4 = attention_gate(x4, x, inter_channel=256)  # Attention gate
    x = layers.concatenate([x, x4])
    x = residual_block(x, 512, l2_reg=l2_reg)

    # Decoder Level 3 (up to Level 2)
    x = layers.Conv3DTranspose(256, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    x3 = attention_gate(x3, x, inter_channel=128)  # Attention gate
    x = layers.concatenate([x, x3])
    x = residual_block(x, 256, l2_reg=l2_reg)

    # Decoder Level 2 (up to Level 1)
    x = layers.Conv3DTranspose(128, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    x2 = attention_gate(x2, x, inter_channel=64)  # Attention gate
    x = layers.concatenate([x, x2])
    x = residual_block(x, 128, l2_reg=l2_reg)

    # Decoder Level 1 (up to original resolution)
    x = layers.Conv3DTranspose(64, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    x1 = attention_gate(x1, x, inter_channel=32)  # Attention gate
    x = layers.concatenate([x, x1])
    x = residual_block(x, 64, l2_reg=l2_reg)

    # Output Layer
    outputs = layers.Conv3D(num_classes, kernel_size=(1, 1, 1), activation='softmax', padding='same')(x)

    # Create the model
    model = models.Model(inputs, outputs)

    return model
